In [128]:
import string
import re
import os
import tempfile
import logging
from gensim import corpora
from gensim import models
from gensim.corpora import Dictionary
numberOfTopics = 200

In [129]:
table = open("../data/paperTable.tsv","r")
entries = []
for line in table:
    entries.append(line.split('\t'))
table.close()

In [130]:
# Read in abstract year of publication, title of abstract, and abstract text
abstracts = []
titleOfAbstract = []
yearOfAbstract = []
for articles in entries:
    titleOfAbstract.append(articles[0])
    abstracts.append(articles[1]+articles[2]+articles[3])
    yearOfAbstract.append(articles[4][:-1])

In [131]:
# Create a set of frequent words
stopFile = open("../data/stopwords.txt","r")
stopWords = stopFile.read().splitlines()
stopWords.append("\xc2\xa9") #This is the copyright symbol, this shows up in every abstract and should not be apart of the corpus
stopWords.extend(["\u2019","\u03bc","bee","bees","honey","honeybee","honeybees"])
stopList = set(stopWords)

# Lowercase each document, split it by white space and filter out stopWords
texts = []
for document in abstracts:
    docwords = []
    for word in document.lower().split():
        word = re.sub(r'[^\w\s]','',word)
        word = re.sub(r'\.+$','',word)
        isNumber = re.compile('^[0-9]+$')
        if isNumber.search(word):
            word = ''
        if word not in stopList and word!='':
            docwords.append(word)
    texts.append(docwords)

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
        
processedCorpus = [[token for token in text if frequency[token] > 5] for text in texts]

In [132]:
# Save the dictionary of tokens
tempFolder = tempfile.gettempdir()
dictionary = corpora.Dictionary(processedCorpus)
dictionary.save(os.path.join(tempFolder,'words.dict'))

2018-05-23 22:42:29,434 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-05-23 22:42:29,706 : INFO : built Dictionary(3860 unique tokens: [u'limited', u'represent', u'believed', u'alleles', u'copy']...) from 1044 documents (total 148749 corpus positions)
2018-05-23 22:42:29,709 : INFO : saving Dictionary object under c:\users\user\appdata\local\temp\words.dict, separately None
2018-05-23 22:42:29,717 : INFO : saved c:\users\user\appdata\local\temp\words.dict


In [133]:
# Create general corpus and serialize in order for it to be iterated over
corpus = [dictionary.doc2bow(text) for text in processedCorpus]
corpora.MmCorpus.serialize(os.path.join(tempFolder, 'words.dict'), corpus)

2018-05-23 22:42:30,043 : INFO : storing corpus in Matrix Market format to c:\users\user\appdata\local\temp\words.dict
2018-05-23 22:42:30,046 : INFO : saving sparse matrix to c:\users\user\appdata\local\temp\words.dict
2018-05-23 22:42:30,048 : INFO : PROGRESS: saving document #0
2018-05-23 22:42:30,624 : INFO : PROGRESS: saving document #1000
2018-05-23 22:42:30,657 : INFO : saved 1044x3860 matrix, density=2.473% (99651/4029840)
2018-05-23 22:42:30,661 : INFO : saving MmCorpus index to c:\users\user\appdata\local\temp\words.dict.index


The above corpus shows the amount of times every word used in the documents is used in every indevidual document. Every word is represented by a token ID, the list of which can be found in "words.dict"

In [ ]:
# Train the model and set number of topics
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
lda = models.ldamodel.LdaModel(corpus,id2word=dictionary,num_topics=numberOfTopics)

2018-05-23 22:42:33,436 : INFO : using symmetric alpha at 0.005
2018-05-23 22:42:33,440 : INFO : using symmetric eta at 0.005
2018-05-23 22:42:33,444 : INFO : using serial LDA version on this node
2018-05-23 22:42:33,607 : INFO : running online (single-pass) LDA training, 200 topics, 1 passes over the supplied corpus of 1044 documents, updating model once every 1044 documents, evaluating perplexity every 1044 documents, iterating 50x with a convergence threshold of 0.001000
2018-05-23 22:42:33,608 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


In [ ]:
# Sort the most interesting words per topic per document
# This cell does not need to be run if only trying to create Top Nine terms per paper
topicOrganizingFile = open("../data/topicorganization.tsv","w")
for x in xrange(0,len(abstracts)):
    doc = dictionary.doc2bow(abstracts[x].split())
    docTopics, wordTopics, phiValues = lda.get_document_topics(doc, per_word_topics=True)
    topicOrganizingFile.write(yearOfAbstract[x]+"\t"+titleOfAbstract[x]+"\t")
    for y in xrange(0,min(3,len(docTopics))):
        topicnumber = docTopics[y][0]
        topicOrganizingFile.write(str(lda.show_topic(topicnumber))+"\t")
        #Sorts the word topics in decending order based on their greatest phi value
        for z in xrange(0,len(phiValues)):
            phiValues[z][1].sort(key=lambda q:q[1],reverse=True)
        phiValues.sort(key=lambda q:q[1][0][1],reverse=True)
        curindex=0
        topwords = ""
        for z in xrange(0,3):
            while curindex<len(phiValues) and phiValues[curindex][1][0][0]!=topicnumber:
                curindex+=1
            if(curindex>=len(phiValues)):break
            print len(phiValues)
            print dictionary[phiValues[curindex][0]]
            topwords+=str(dictionary[phiValues[curindex][0]].encode('utf-8').strip())+" "
            curindex+=1
        filter(lambda a:a[0]!=topicnumber,phiValues)
        topicOrganizingFile.write(topwords+"\t")
    topicOrganizingFile.write("\n")
topicOrganizingFile.close()

        

In [ ]:
topicWords = []
for i in range(0,numberOfTopics):
    t = lda.get_topic_terms(i,50)
    currentWordList = []
    for x in t:
        word = str(dictionary[x[0]])
        if word not in currentWordList:
            currentWordList.append(word)
    topicWords.append(currentWordList)
topicListFile = open("../data/TopicWords/List-"+str(numberOfTopics)+".txt","w+")
for i in range(0,len(topicWords)):
    topicListFile.write("Topic "+str(i)+":\n")
    for j in topicWords[i]:
        topicListFile.write(j+'\n')
    topicListFile.write('\n')
topicListFile.close()

In [ ]:
#Makes the top nine terms for each document

topNineFile = open("../data/Docbow/TopNineTerms-"+str(numberOfTopics)+".tsv","w")
for x in xrange(0,len(abstracts)):
    doc = dictionary.doc2bow(abstracts[x].split()) # Convert to bag of words format first
    # Get the topics and words associated with each document
    docTopics, wordTopics, phiValues = lda.get_document_topics(doc, per_word_topics=True)
    topNineFile.write(yearOfAbstract[x]+"\t"+titleOfAbstract[x]+"\t")
    for z in xrange(0,len(phiValues)):
        phiValues[z][1].sort(key=lambda q:q[1],reverse=True)
    phiValues.sort(key=lambda q:q[1][0][1],reverse=True)
    nineWords = ""
    for x in phiValues[:15]:
        nineWords+= dictionary[x[0]] + " "
    topNineFile.write(nineWords.encode('utf-8')+"\n")

In [45]:
#Makes the top 3 terms for the top 5 documents
#
#with open('../data/Docbow/TopFifteenFor'+str(numberOfTopics)+'1.tsv','w') as top15:
    t = lda.print_topics(num_topics=numberOfTopics,num_words=20)
    for topic in range(0,5):
        top15.write(str(t[topic][1].split("+")[0:3])+"\n")


2018-05-23 21:50:15,805 : INFO : topic #0 (0.005): 0.013*"mellifera" + 0.011*"colonies" + 0.010*"apis" + 0.010*"species" + 0.009*"population" + 0.009*"genetic" + 0.006*"varroa" + 0.006*"effects" + 0.006*"colony" + 0.005*"food" + 0.005*"recombination" + 0.005*"control" + 0.004*"nosema" + 0.004*"imidacloprid" + 0.004*"gene" + 0.004*"animals" + 0.004*"animal" + 0.004*"also" + 0.004*"proteins" + 0.004*"analysis"
2018-05-23 21:50:15,809 : INFO : topic #1 (0.005): 0.010*"colony" + 0.010*"pollen" + 0.009*"effects" + 0.009*"colonies" + 0.008*"mellifera" + 0.008*"exposure" + 0.007*"summer" + 0.007*"apis" + 0.006*"concentration" + 0.006*"races" + 0.006*"pathogens" + 0.006*"spring" + 0.005*"toxicity" + 0.005*"osmotic" + 0.005*"losses" + 0.005*"three" + 0.005*"environmental" + 0.005*"seasons" + 0.005*"insecticides" + 0.004*"diseases"
2018-05-23 21:50:15,812 : INFO : topic #2 (0.005): 0.014*"pollen" + 0.012*"virus" + 0.009*"apis" + 0.008*"nosema" + 0.007*"exposure" + 0.007*"colonies" + 0.007*"larva

2018-05-23 21:50:15,868 : INFO : topic #20 (0.005): 0.010*"queens" + 0.009*"viability" + 0.009*"colony" + 0.009*"species" + 0.008*"two" + 0.008*"sperm" + 0.007*"health" + 0.007*"mellifera" + 0.007*"effects" + 0.006*"queen" + 0.006*"insecticides" + 0.006*"apis" + 0.005*"high" + 0.005*"temperature" + 0.005*"colonies" + 0.005*"animal" + 0.004*"data" + 0.004*"pollen" + 0.004*"mite" + 0.004*"gene"
2018-05-23 21:50:15,869 : INFO : topic #21 (0.005): 0.017*"expression" + 0.015*"gene" + 0.013*"nosema" + 0.011*"ceranae" + 0.010*"pesticide" + 0.009*"apis" + 0.009*"rna" + 0.008*"animal" + 0.008*"pollen" + 0.008*"parasite" + 0.007*"pesticides" + 0.007*"n" + 0.007*"mellifera" + 0.007*"disease" + 0.007*"treatment" + 0.007*"host" + 0.006*"genes" + 0.006*"immune" + 0.006*"effects" + 0.006*"matrices"
2018-05-23 21:50:15,872 : INFO : topic #22 (0.005): 0.027*"virus" + 0.013*"colony" + 0.010*"colonies" + 0.008*"immune" + 0.007*"study" + 0.007*"animal" + 0.007*"destructor" + 0.007*"varroa" + 0.006*"winter

2018-05-23 21:50:15,927 : INFO : topic #40 (0.005): 0.010*"apis" + 0.008*"environmental" + 0.008*"effects" + 0.007*"virus" + 0.007*"mellifera" + 0.006*"may" + 0.006*"colony" + 0.006*"indigenous" + 0.006*"data" + 0.006*"nosema" + 0.006*"species" + 0.006*"research" + 0.005*"results" + 0.005*"sequence" + 0.004*"ceranae" + 0.004*"development" + 0.004*"crops" + 0.004*"animals" + 0.004*"exposure" + 0.004*"analysis"
2018-05-23 21:50:15,930 : INFO : topic #41 (0.005): 0.015*"mites" + 0.014*"varroa" + 0.011*"nosema" + 0.011*"colonies" + 0.010*"mite" + 0.009*"apis" + 0.008*"n" + 0.008*"mellifera" + 0.007*"brood" + 0.006*"host" + 0.006*"colony" + 0.006*"analysis" + 0.006*"exposed" + 0.005*"ceranae" + 0.005*"exposure" + 0.005*"selection" + 0.005*"days" + 0.005*"species" + 0.005*"resistance" + 0.005*"infestation"
2018-05-23 21:50:15,933 : INFO : topic #42 (0.005): 0.009*"apis" + 0.009*"colony" + 0.009*"mellifera" + 0.009*"mite" + 0.006*"destructor" + 0.006*"used" + 0.006*"effect" + 0.006*"health" +

2018-05-23 21:50:15,993 : INFO : topic #60 (0.005): 0.012*"pollen" + 0.011*"effect" + 0.010*"dch" + 0.010*"species" + 0.009*"acid" + 0.008*"diversity" + 0.008*"kunkeei" + 0.008*"larvae" + 0.008*"protein" + 0.007*"also" + 0.006*"apis" + 0.006*"diet" + 0.006*"mellifera" + 0.006*"food" + 0.006*"exposure" + 0.006*"content" + 0.006*"potential" + 0.006*"oxalic" + 0.006*"mexico" + 0.006*"pesticide"
2018-05-23 21:50:15,996 : INFO : topic #61 (0.005): 0.010*"losses" + 0.009*"health" + 0.009*"colony" + 0.009*"nosema" + 0.009*"mellifera" + 0.008*"breeding" + 0.007*"apis" + 0.006*"programs" + 0.006*"animal" + 0.006*"effects" + 0.006*"europe" + 0.006*"review" + 0.006*"management" + 0.006*"colonies" + 0.006*"behavior" + 0.006*"pesticides" + 0.005*"use" + 0.005*"ceranae" + 0.005*"effect" + 0.005*"grooming"
2018-05-23 21:50:15,999 : INFO : topic #62 (0.005): 0.033*"virus" + 0.012*"viruses" + 0.009*"colonies" + 0.009*"paralysis" + 0.008*"gene" + 0.007*"colony" + 0.007*"apis" + 0.006*"mellifera" + 0.006

2018-05-23 21:50:16,059 : INFO : topic #80 (0.005): 0.010*"varroa" + 0.009*"colonies" + 0.007*"mellifera" + 0.007*"species" + 0.007*"colony" + 0.006*"larvae" + 0.006*"pollen" + 0.006*"sequence" + 0.006*"apis" + 0.005*"kbv" + 0.005*"pesticide" + 0.005*"health" + 0.005*"effects" + 0.005*"lab" + 0.005*"may" + 0.005*"insecticides" + 0.005*"virus" + 0.005*"used" + 0.005*"rna" + 0.005*"control"
2018-05-23 21:50:16,062 : INFO : topic #81 (0.005): 0.011*"food" + 0.008*"gene" + 0.008*"virus" + 0.008*"queens" + 0.007*"expression" + 0.007*"production" + 0.007*"industry" + 0.007*"population" + 0.007*"sustainability" + 0.006*"colony" + 0.006*"apis" + 0.006*"animal" + 0.006*"genes" + 0.006*"chemical" + 0.006*"social" + 0.006*"development" + 0.005*"immune" + 0.005*"pollen" + 0.005*"environmental" + 0.004*"analysis"
2018-05-23 21:50:16,065 : INFO : topic #82 (0.005): 0.029*"loss" + 0.022*"colony" + 0.020*"heart" + 0.018*"colonies" + 0.012*"octopamine" + 0.012*"mellifera" + 0.011*"average" + 0.009*"api

2018-05-23 21:50:16,121 : INFO : topic #100 (0.005): 0.011*"colony" + 0.008*"mellifera" + 0.007*"varroa" + 0.007*"pollen" + 0.007*"cell" + 0.006*"rna" + 0.006*"sequence" + 0.006*"pesticides" + 0.006*"may" + 0.005*"losses" + 0.005*"apis" + 0.005*"dna" + 0.005*"effects" + 0.005*"analysis" + 0.004*"study" + 0.004*"virus" + 0.004*"gene" + 0.004*"conditions" + 0.004*"genetic" + 0.004*"using"
2018-05-23 21:50:16,124 : INFO : topic #101 (0.005): 0.010*"pollen" + 0.007*"protein" + 0.007*"sequence" + 0.007*"acid" + 0.007*"presence" + 0.007*"gene" + 0.006*"insect" + 0.006*"apis" + 0.005*"dna" + 0.005*"animal" + 0.005*"study" + 0.005*"analysis" + 0.005*"colonies" + 0.005*"mellifera" + 0.005*"behavior" + 0.005*"oxalic" + 0.004*"control" + 0.004*"varroa" + 0.004*"destructor" + 0.004*"species"
2018-05-23 21:50:16,127 : INFO : topic #102 (0.005): 0.016*"colonies" + 0.014*"colony" + 0.014*"ci" + 0.013*"losses" + 0.012*"winter" + 0.012*"beekeepers" + 0.012*"varroa" + 0.009*"mortality" + 0.009*"rates" +

2018-05-23 21:50:16,197 : INFO : topic #121 (0.005): 0.011*"colony" + 0.010*"land" + 0.008*"pollen" + 0.007*"use" + 0.007*"colonies" + 0.007*"apis" + 0.007*"mellifera" + 0.007*"cover" + 0.006*"effects" + 0.006*"maps" + 0.006*"ecosystem" + 0.005*"health" + 0.005*"ccd" + 0.005*"varroa" + 0.005*"virus" + 0.005*"two" + 0.005*"also" + 0.004*"propolis" + 0.004*"animal" + 0.004*"nosema"
2018-05-23 21:50:16,201 : INFO : topic #122 (0.005): 0.019*"pollen" + 0.014*"effects" + 0.013*"aspergillus" + 0.010*"pollens" + 0.009*"diet" + 0.008*"neonicotinoid" + 0.007*"drug" + 0.007*"colonies" + 0.007*"fumigatus" + 0.006*"derivative" + 0.006*"animal" + 0.006*"effect" + 0.006*"interactions" + 0.006*"used" + 0.006*"parasite" + 0.005*"colony" + 0.005*"reproduction" + 0.005*"toxicity" + 0.005*"dna" + 0.005*"animals"
2018-05-23 21:50:16,204 : INFO : topic #123 (0.005): 0.013*"tradeoffs" + 0.012*"dcas" + 0.010*"virus" + 0.009*"infection" + 0.008*"pesticide" + 0.008*"locations" + 0.007*"drones" + 0.007*"risk" +

2018-05-23 21:50:16,259 : INFO : topic #142 (0.005): 0.026*"gis" + 0.020*"residues" + 0.013*"pesticide" + 0.012*"pesticides" + 0.012*"spatial" + 0.009*"samples" + 0.007*"wild" + 0.007*"research" + 0.007*"males" + 0.007*"used" + 0.007*"apis" + 0.006*"ppm" + 0.006*"analyses" + 0.006*"time" + 0.006*"decrease" + 0.006*"will" + 0.006*"geographic" + 0.006*"paper" + 0.005*"light" + 0.005*"pollen"
2018-05-23 21:50:16,260 : INFO : topic #143 (0.005): 0.011*"effects" + 0.009*"imidacloprid" + 0.007*"toxicity" + 0.007*"concentrations" + 0.007*"gene" + 0.007*"insecticides" + 0.007*"significantly" + 0.006*"derivative" + 0.006*"study" + 0.006*"may" + 0.006*"however" + 0.006*"high" + 0.005*"pesticides" + 0.005*"mortality" + 0.005*"animals" + 0.005*"environmental" + 0.005*"insecticide" + 0.005*"chlorpyrifos" + 0.005*"symbiont" + 0.005*"drug"
2018-05-23 21:50:16,263 : INFO : topic #144 (0.005): 0.011*"species" + 0.011*"exposure" + 0.010*"pollen" + 0.010*"plant" + 0.009*"mellifera" + 0.008*"pesticide" + 

2018-05-23 21:50:16,316 : INFO : topic #162 (0.005): 0.022*"pollen" + 0.011*"food" + 0.011*"thymol" + 0.010*"diet" + 0.009*"development" + 0.007*"fungi" + 0.007*"acid" + 0.007*"plant" + 0.007*"growth" + 0.006*"antifungal" + 0.006*"larval" + 0.005*"aspergillus" + 0.005*"survival" + 0.005*"penicillium" + 0.005*"drones" + 0.005*"mellifera" + 0.005*"species" + 0.005*"molecular" + 0.005*"study" + 0.005*"insecticidal"
2018-05-23 21:50:16,321 : INFO : topic #163 (0.005): 0.020*"losses" + 0.016*"colony" + 0.016*"winter" + 0.011*"colonies" + 0.009*"factors" + 0.009*"virus" + 0.008*"different" + 0.006*"management" + 0.006*"study" + 0.006*"health" + 0.006*"regions" + 0.006*"bombus" + 0.005*"apiaries" + 0.005*"practices" + 0.005*"argentina" + 0.005*"collapse" + 0.005*"higher" + 0.004*"data" + 0.004*"pollen" + 0.004*"rna"
2018-05-23 21:50:16,322 : INFO : topic #164 (0.005): 0.009*"colony" + 0.009*"colonies" + 0.008*"pollen" + 0.008*"development" + 0.008*"virus" + 0.006*"gene" + 0.006*"apis" + 0.006

2018-05-23 21:50:16,381 : INFO : topic #182 (0.005): 0.022*"virus" + 0.019*"colony" + 0.016*"rna" + 0.010*"colonies" + 0.010*"risk" + 0.010*"mites" + 0.008*"apis" + 0.008*"varroa" + 0.007*"collapse" + 0.007*"mellifera" + 0.006*"gene" + 0.006*"associated" + 0.006*"important" + 0.006*"mite" + 0.006*"beekeeping" + 0.006*"population" + 0.005*"states" + 0.005*"brood" + 0.005*"found" + 0.005*"immune"
2018-05-23 21:50:16,384 : INFO : topic #183 (0.005): 0.010*"apis" + 0.009*"mellifera" + 0.009*"exposure" + 0.009*"effects" + 0.008*"animal" + 0.008*"pesticide" + 0.007*"thiamethoxam" + 0.007*"colony" + 0.007*"gene" + 0.007*"animals" + 0.006*"pollen" + 0.006*"derivative" + 0.006*"behavior" + 0.006*"colonies" + 0.006*"health" + 0.005*"development" + 0.005*"residue" + 0.005*"results" + 0.005*"study" + 0.005*"larva"
2018-05-23 21:50:16,387 : INFO : topic #184 (0.005): 0.009*"pesticides" + 0.008*"entomological" + 0.007*"new" + 0.007*"may" + 0.007*"varroa" + 0.007*"growth" + 0.006*"zealand" + 0.006*"p

In [127]:
top5 = lda.top_topics(corpus=corpus,topn=25)[0:5]
top15 = []
for t in top5:
    wordsAdded = 0
    temp = set([])
    for w in t[0][0:10]:
        temp.add(w[1])
    top15.append(temp)
output = []
with open('../data/Docbow/TopFifteenFor'+str(numberOfTopics)+'.tsv','w') as fileToWrite:
    for wordSet in top15:
        wordsAdded = 0
        for word in wordSet:
            if word not in output and wordsAdded < 3:
                output.append(word)
                fileToWrite.write(word+",")
                wordsAdded += 1
            elif wordsAdded == 3:
                break
        fileToWrite.write("\n")

2018-05-23 22:41:32,640 : INFO : CorpusAccumulator accumulated stats from 1000 documents
